In [1]:
from importlib import resources
import sys
import inspect
from pathlib import Path
import traceback
import tempfile, os, shutil, time
import signal

from kinoml.core.ligands import Ligand
from kinoml.core.proteins import Protein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import (
    MostSimilarPDBLigandFeaturizer,
    OEDockingFeaturizer,
)

from rdkit import Chem
import pandas as pd
import requests as req
from multiprocessing import Pool, cpu_count
import tqdm
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import LocalCluster, Client

In [2]:
NPARTITION = 10
TIMEOUT = 120  # in sec

# directories
HERE = Path(".").absolute()
CACHE_DIR = HERE / "docking_pipeline"
MOST_SIMILAR = CACHE_DIR / "most_similar.csv"
KLIFS_DIR = CACHE_DIR / "KLIFS"
KLIFS_MAP = CACHE_DIR / "similar_klifs_structures.csv"
TEMP_DIR = CACHE_DIR / "temporary"
DOCKING_DIR = CACHE_DIR / "docking"

In [3]:
kinodata = pd.read_csv("data/activities-chembl31.csv", index_col=0)

In [4]:
kinodata = kinodata.head(5)

# Find most similar PDBs

In [5]:
def get_most_similar(ident, uniprot_id, ligand_smiles):
    protein = Protein(uniprot_id=uniprot_id, toolkit="MDAnalysis")
    ligand = Ligand(smiles=ligand_smiles)
    system = ProteinLigandComplex(components=[protein, ligand])

    featurizer = MostSimilarPDBLigandFeaturizer(
        similarity_metric="fingerprint",
        cache_dir=CACHE_DIR,
    )

    system = featurizer.featurize([system])[0]

    ligand_id = system.protein.expo_id
    pdb_id = system.protein.pdb_id
    chain = system.protein.chain_id

    with open(MOST_SIMILAR, "a") as f:
        f.write(",".join(map(str, [ident, ligand_id, pdb_id, chain])) + "\n")
    return 1

In [6]:
with open(MOST_SIMILAR, "w") as f:
    f.write(
        "activities.activity_id,similar.ligand_pdb,similar.complex_pdb,similar.chain\n"
    )

In [7]:
similars = list()
for _, row in kinodata.iterrows():
    uniprot_id = row["UniprotID"]
    ligand_smiles = row["compound_structures.canonical_smiles"]
    ident = row["activities.activity_id"]

    # most similar PDB
    similars.append(get_most_similar(ident, uniprot_id, ligand_smiles))

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 30, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 29, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 34, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 67, aromatic: False, chiral: False

War

# Download KLIFS structures

In [8]:
ms = pd.read_csv(MOST_SIMILAR)

In [9]:
kinodata = ms.merge(kinodata, how="inner", on="activities.activity_id")

In [10]:
# kinodata = dd.from_pandas(kinodata, npartitions=NPARTITION)

In [11]:
def klifs_pdb(
    ident,
    pdb_id: str,
    chain: str,
    ligand_id: str,
    output_path=Path(".").absolute(),
    lazy: bool = False,
):
    """
    Get the complex PDB from KLIFS.

    Parameters
    ----------
    pdb_id: str
        PDB ID.
    chain: str
        the chain.
    ligand_id: str
        ligand PDB ID
    path: Path, optional
        folder to store the structure in.

    Returns
    -------
    file_path: Path
        path of the PDB file.
    """
    resp = req.get(
        "https://klifs.net/api_v2/structures_pdb_list", {"pdb-codes": pdb_id}
    )
    klifs_info = None
    resp.raise_for_status()
    for info in resp.json():
        if (
            str(info["chain"]).upper() == str(chain).upper()
            and str(info["ligand"]).upper() == str(ligand_id).upper()
        ):
            if klifs_info is None:
                klifs_info = info
            elif klifs_info["quality_score"] < info["quality_score"]:
                klifs_info = info
    if klifs_info is None:
        raise ValueError(f"not found pdb:{pdb_id} chain:{chain} lig_pdb:{ligand_id}")
    structure_ID = klifs_info["structure_ID"]
    output_path.mkdir(exist_ok=True)
    filename = output_path / f"{structure_ID}.pdb"
    if not lazy or not filename.exists():
        resp = req.get(
            "https://klifs.net/api_v2/structure_get_pdb_complex",
            {"structure_ID": structure_ID},
        )

    with open(filename, "w") as f:
        f.write(resp.text)

    with open(KLIFS_MAP, "a") as f:
        f.write(f"{ident},{structure_ID}\n")

    return 0

In [12]:
with open(KLIFS_MAP, "w") as f:
    f.write("activities.activity_id,similar.klifs_structure_id\n")

In [13]:
jobs = list()
for _, row in kinodata.iterrows():
    pdb_id = row["similar.complex_pdb"]
    ligand_pdb = row["similar.ligand_pdb"]
    chain = row["similar.chain"]
    ident = row["activities.activity_id"]
    jobs.append(
        klifs_pdb(ident, pdb_id, chain, ligand_pdb, output_path=KLIFS_DIR, lazy=False)
    )

# OpenEye Template docking

In [14]:
klifs_map = pd.read_csv(KLIFS_MAP)
kinodata = klifs_map.merge(kinodata, how="inner", on="activities.activity_id")

In [15]:
class temporary_copy(object):
    def __init__(self, original_path):
        self.original_path = original_path

    def __enter__(self):
        temp_dir = tempfile.gettempdir()
        base_path = os.path.basename(self.original_path)
        self.path = os.path.join(temp_dir, base_path)
        shutil.copy2(self.original_path, self.path)
        return self.path

    def __exit__(self, exc_type, exc_val, exc_tb):
        os.remove(self.path)

In [16]:
def handler(signum, frame):
    print("Stop it already!")
    raise Exception("timeout")

In [17]:
# @dask.delayed
def posit(ident, pdb_filepath, ligand_smiles, output_dir):
    print(pdb_filepath)
    protein = Protein.from_file(pdb_filepath)
    ligand = Ligand(smiles=ligand_smiles)
    system = ProteinLigandComplex(components=[protein, ligand])
    featurizer = OEDockingFeaturizer(
        output_dir=output_dir, method="Posit", use_multiprocessing=False
    )

    signal.signal(signal.SIGALRM, handler)
    signal.alarm(TIMEOUT)
    try:
        system = featurizer.featurize([system])[0]
    except Exception as exc:
        print(exc)
        return 1

    print("write result")
    universe = system.featurizations["last"]
    docking_score = universe._topology.docking_score
    posit_probability = universe._topology.posit_probability

    print("write", output_dir / f"{activity_id}_complex.pdb")
    with mda.coordinates.PDB.PDBWriter(output_dir / f"{activity_id}_complex.pdb") as w:
        w.write(universe)
    with mda.coordinates.PDB.PDBWriter(output_dir / f"{activity_id}_ligand.pdb") as w:
        w.write(universe.select_atoms("resname LIG"))

    with open(output_dir / "docking.csv", "a") as f:
        f.write(
            ",".join(
                list(map(str, [ident, docking_score, posit_probability, duration]))
            )
            + "\n"
        )
    return 0

In [ ]:
jobs = list()
for _, row in kinodata.iterrows():
    ident = row["activities.activity_id"]
    pdb_filepath = KLIFS_DIR / (str(row["similar.klifs_structure_id"]) + ".pdb")
    ligand_smiles = row["compound_structures.canonical_smiles"]
    jobs.append(posit(ident, pdb_filepath, ligand_smiles, DOCKING_DIR))

/home/michael/Code/kinodata-docking/docking_pipeline/KLIFS/12757.pdb


OEDockingFeaturizer:   0%|          | 0/1 [00:00<?, ?it/s]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 30, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 29, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 34, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 67, aromatic: False, chiral: False

